In [1]:
import pandas as pd

# Sales Forecast Prediction
This task uses a dataset for a large Austrailian company Austral AB with sales data from a 1009 of stores in 20 fictitious towns.

## Problem Description
In uncertain business times, Austral AB would love to have predictable revenue. The revenue is often the starting point for annual plans and budgets affecting the whole organization. Now after the last expansion 2009, where they expanded from 250 to 1009 stores, and with distributed sales teams the future growth strategy highly rely on revenue forecasting.

With sales forecasting they woud be able to:
- Make better business decisions for business planning, budgeting and risk management.
- Efficiently allocate resources for future growth and cash flow management.
- Efficiently mange the sales teams achieve their goals by preventative instead of reactive management. 
- Estimate costs and revenue accurately, and therefore ability to predict the companys short- and long-term performance.

However, they noticed that their current short term forecasting is way off the reality and would therfore like some help to predict number of customers and revenue. 


## Objective
To predict the number of customers and the sales figures for the time period that follows directly after the time period covered in the dataset (2012-01-01 - 2012-02-03).


## Data

### Files
- Train model: `sales_data.csv` (~ 900.000 entries)
- Produce predictions on `sales_data_forecast.csv` (~ 35.000 entries)
- Deliver `sales_data_forecast.csv` with prediction of `n_cust` and `revenue`

### Features
- `store_id`: An integer ID for each store 
- `date`: The date in year-month-day format (string)
- `is_store_open`: Binary variable indicating if the store is open on this date or not (1 = open)
- `is_sale_period`: Binary variable indicating if the store has an ongoing sale (1 = yes)
- `town`: Name of the (fictitious) town


### Target Variables
- `n_cust`: The number of customers who visited the store on the given date
- `revenue`: The total revenue that was earned by the store on the given date

# Exploratory Data Analysis

## Business Context
### Training Data
- Number of stores: 1009
- Number of towns: 20
- Dates: 2009-07-07 - 2011-12-31
- Nr of days: 908
- Percent open: 82.9 %
- Percent sale period: 37.92 %
- Number of customers: 0 - 10065 (mean 778)
- Revenue: 0 - 70935  (mean 7099)
    

### Test Data
- Number of stores: 1009
- Number of towns: 20
- Dates: 2012-01-01 - 2012-02-03
- Nr of days: 34
- Percent open: 85.57 %
- Percent sale period: 44.12 %

## Univariate Analysis
<img src="./visuals/histogram_town.png">
<img src="./visuals/histogram_n_cust.png">
<img src="./visuals/histogram_revenue.png">   
  
    
      
        
        

## Bivariate Analysis

### Aggregated Time Series Year (2011)
<img src="./visuals/timeserie_n_cust_2011-01-01_2011-12-31.png">
<img src="./visuals/timeserie_revenue_2011-01-01_2011-12-31.png">

### Aggregated Time Series Month (Mar 2011)
<img src="./visuals/timeserie_n_cust_2011-03-01_2011-03-31.png">
<img src="./visuals/timeserie_revenue_2011-03-01_2011-03-31.png">

### Responses Scatterplot
<img src="./visuals/scatterplot_responses.png">

### Revenues per Town
<img src="./visuals/boxplot_revenue_store.png">


## Correlations
<img src="./visuals/corr_coef.png">

# Forecasting

## Considerations
The data consists of 1009 stores, which every day reports a nr of features, for about 900 days. 

### Alternatives
The forecast of n_cust and revenue could be considered as a time series or as a regression problem. 

- **TimeSeries**: Uni- and multivariate time series analysis methods were considered because of feature time dependency. For time series models we need to create 1009 models, one for each store, which is very difficult to maintain in production.
    - The _univariate methods_ requires a unique date as index and would be possible to predict total n_cust and revenue for each date and without considering other features. Hence, we loose lots of information. 
    - The _multivariate method VAR (Vector AutoRegression)_ requires a unique date as index and would therefore help us to predict total n_cust and revenue for each date but not for each store.  
    

- **Regression**: The regresson models can deliver the required output.
    - _Linear Regression (with PCA)_: Linear Regression problems require the data to have independent features, which we dont fulfill.
    - _Ensemble Methods - XGBRegressor_: Does not require independent features but are likely to overfit and it's not sure if the results are interpolatable (which we would like for forecasting).

### Model Decision
1. None of the methods are perfect for our purpose, at least the regression methods can produce the required outcome. 
2. Of the regression methods, I tested both to evalute their performance. **XGBoost** was the best performer for short-term forecasting.

## Computational Resources
- MacBook Pro (13-inch, 2019, Four Thunderbolt 3 ports) 
- Processor 2,4 GHz Quad-Core Intel Core i5
- Memory 8 GB 2133 MHz LPDDR3
- Graphics Intel Iris Plus Graphics 655 1536 MB

## Methodology

### Cleaning
Aside from a few outliers, the dataset was clean and tidy already. 
- No duplicates
- No missing values

### Preprocessing
#### Feature Engineering
Features were extracted for ordinal categorical values of:
- Year
- Month
- Week of year
- Week of month
- Quarter
- Season   

The original date column was removed.  
    
      
      
  

#### One-Hot-Encoding
One-Hot-Encoding was performed (although it is not required for tree based methods) for `store_id` and `town`. the original columns for store_id and town was removed.

#### Feature Scaling
Feature scaling was not required for our data, which was ordinal categorical data.  
  
    
    


### Data Split
First of all, due to performace issues with limited memory, the dataset was reduced to ~ 50 % of the original data by choosing the month between sep and feb. Therefore, the model is only valid for these months.

The split into train and test data for the model was done by assigning ~ 10 % of the last consecutive dates into test data. In this way, all stores and all months were included in the training.

Overfitting is a risk as no cross validation is performed. For 3 fold cross validation data could be divided into folds of each each year, with the least month (dec) as test set.

### Model XGBRegressor
The XGBRegressor model was used for both responses, because of high correlation between the responses. Additinally, values predicted below 0 were adjusted to 0.

#### Scatterplot Predictions
<img src="./visuals/scatterplot_xgb_ncust.png">
<img src="./visuals/scatterplot_xgb_revenue.png">

#### Important Features
<img src="./visuals/topfeats_XGBRegressor_n_cust.png">
<img src="./visuals/topfeats_XGBRegressor_revenue.png">
  
    
      
      

#### Hyperparameter Settings  
The model has default hyperparameter settings and is not yet optimized on the hyper parameters. 

## Evaluation
The tested models are stored in an evaluation matrix with model name, hyper parameter setting, MAE and RMSE. 

The model performance are evaluated on Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE).



Results for n_cust predictions

In [2]:
pd.read_csv('./data/modelling_results_mae.csv', index_col=0)

,model,parameters,MAE,RMSE
0,Lasso,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",272.480000,382.750000
1,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",170.260129,233.003534
2,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",359.456236,503.673256
3,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",361.035418,504.523145
4,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",167.816365,232.785170
5,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",173.736974,246.131963


Results for revenue predictions

In [3]:
pd.read_csv('./data/modelling_results_mae_revenue.csv', index_col=0)

,model,parameters,MAE,RMSE
0,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",1565.537094,2245.944976
1,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",1534.648491,2224.810586


### Usefulness
**XGBRegressor for n_cust**  
Average n_cust: 773.3 & average MAE: 173.21 -> 22.4% error 
The prediction has less than average error for 677 stores  
The prediction has higher than average error for 332 stores

**XGBRegressor for revenue**
Average n_cust: 7048.17 & average MAE: 1529.29 -> 22.4% error
The prediction has less than average error for 636 stores
The prediction has higher than average error for 373 stores

As additional output .csv-files are created for decision making of which stores the model is useful for. See below for n_cust and revenue.


In [4]:
ncust_errors = pd.read_csv('./data/results_comparison_ncust.csv', index_col=0).head()
ncust_errors[['store_id', 'percentage_error']]

,store_id,percentage_error
0,1,0.231043
1,2,0.219152
2,3,0.144744
3,4,0.187670
4,5,0.353193


In [5]:
rev_errors = pd.read_csv('./data/results_comparison_revenue.csv', index_col=0).head()
rev_errors[['store_id', 'percentage_error']]


,store_id,percentage_error
0,1,0.367086
1,2,0.326827
2,3,0.124846
3,4,0.160284
4,5,0.359070


# Conclusion
**Can we produce forecast predictions from n_cust and revenue?**   
Yes, we can.

**Are the model predictions accurate enough to be useful?**
- It depends, are the results better than the current situation?
- To determine for which stores the model can be useful, please define the treashold of error tolerance. 


# Future Suggestions
- Improve model performance: 
    - Investigate the cause of the outliers
    - Investigate skewness of the response variables
    - Add public holidays as a date-feature
    - Investgate feature selection based on correlation coefficients (feature selection by XGBoost feature importance decreased performance).
    - Cross Validation with stratified splitted data
- Use MAPE as evaluation metric for easier interpretation and compare with current manual predictability
- Create a pipeline for production
- Track the model tests and the performance with a ML-tracking tool such as MLflow

# Thank you! 
## Do you have any feedback?

In [ ]:
# to run slideshow 
!jupyter nbconvert sales-forecast-report.ipynb --to slides --post serve --SlidesExporter.reveal_scroll=True


[NbConvertApp] Converting notebook sales-forecast-report.ipynb to slides
[NbConvertApp] Writing 601733 bytes to sales-forecast-report.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/sales-forecast-report.slides.html
Use Control-C to stop this server
